In [1]:
from IPython.display import HTML

from tqdm import trange

import pandas as pd
from pandas import DataFrame

import numpy as np
from numpy import zeros

from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, Descriptors, AllChem
import rdkit.Chem.AllChem as AllChem

from CGRtools import SDFRead
from CGRtools.utils import grid_depict, to_rdkit_molecule

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, balanced_accuracy_score
from sklearn.model_selection import KFold, cross_val_predict, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# создаем словарь из дескриторов структуры
ConstDescriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
                    "NHOHCount": Descriptors.NHOHCount,
                    "NOCount": Descriptors.NOCount,
                    "NumHAcceptors": Descriptors.NumHAcceptors,
                    "NumHDonors": Descriptors.NumHDonors,
                    "NumHeteroatoms": Descriptors.NumHeteroatoms,
                    "NumRotatableBonds": Descriptors.NumRotatableBonds,
                    "NumValenceElectrons": Descriptors.NumValenceElectrons,
                    "NumAromaticRings": Descriptors.NumAromaticRings,
                    "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
                    "RingCount": Descriptors.RingCount}

# создаем словарь из физико-химических дескрипторов                            
PhisChemDescriptors = {"MW": Descriptors.MolWt,
                       "LogP": Descriptors.MolLogP,
                       "MR": Descriptors.MolMR,
                       "TPSA": Descriptors.TPSA}

# объединяем все дескрипторы в один словарь
descriptors = {}
descriptors.update(ConstDescriptors)
descriptors.update(PhisChemDescriptors)
print(f"Количество дескрипторов в словаре: {len(descriptors)}")


# функция для генерации дескрипторов из молекул
def mol_dsc_calc(mols): 
    return DataFrame({k: f(m) for k, f in descriptors.items()} 
             for m in mols)

# оформляем sklearn трансформер для использования в конвеерном моделировании (sklearn Pipeline)
descriptors_transformer = FunctionTransformer(mol_dsc_calc, validate=False)

Количество дескрипторов в словаре: 15


In [3]:
def calc_morgan(mols):
    """ генерация молекулярных отпечатков по методу Моргана с радиусом 2
    """
    for_df = []
    for m in mols:
        arr = zeros((1,), dtype='float32')
        DataStructs.ConvertToNumpyArray(AllChem.GetMorganFingerprintAsBitVect(m, 2, 2048), arr)
        for_df.append(arr)
    return DataFrame(for_df)

Ввиду того, что изначальный датасет является фокусированной библиотекой, необходимо добавить молекул. К сожалению, их получилось не очень много. Добавленных молекул получилось 19144, что в целом более-менее сопоставимо с размером фокусированной библиотеки.

In [4]:
molecules_adenosine = [mol for mol in Chem.SDMolSupplier("Adenosine_set.sdf") if mol is not None]
print(f'Молекул фокусированной библиотеки = {len(molecules_adenosine)}')
print('===')
molecules_covid = [mol for mol in Chem.SDMolSupplier("COVID_set.sdf") if mol is not None]
print(f'Количество молекул = {len(molecules_covid)}')
print('===')
molecules_covid_small = [mol for mol in Chem.SDMolSupplier("COVID_set-small.sdf") if mol is not None]
print(f'Количество молекул = {len(molecules_covid_small)}')
print('===')
molecules_HIV_1 = [mol for mol in Chem.SDMolSupplier("HIV-1_set.sdf") if mol is not None]
print(f'Количество молекул = {len(molecules_HIV_1)}')
print('===')
molecules_HIV_2 = [mol for mol in Chem.SDMolSupplier("HIV-2_set.sdf") if mol is not None]
print(f'Количество молекул = {len(molecules_HIV_2)}')
print('===')
molecules_HIV_3 = [mol for mol in Chem.SDMolSupplier("HIV-3_set.sdf") if mol is not None]
print(f'Количество молекул = {len(molecules_HIV_3)}')

Молекул фокусированной библиотеки = 20323
===
Количество молекул = 3668
===
Количество молекул = 471
===
Количество молекул = 5005
===
Количество молекул = 5000
===
Количество молекул = 5000


In [5]:
molecules = molecules_adenosine + molecules_covid + molecules_covid_small + molecules_HIV_1 + molecules_HIV_2 + molecules_HIV_3

In [6]:
X = descriptors_transformer.transform(molecules)

In [7]:
morgan_transformer = FunctionTransformer(calc_morgan, validate=False)

In [8]:
df = pd.DataFrame(X)

In [9]:
scaler = StandardScaler()
scaler.fit(X.values)
X_norm_SS = DataFrame(scaler.transform(X.values), index=X.index, columns=X.columns)
X_norm_SS

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,0.951010,0.555074,0.995655,0.477653,0.846574,0.859757,2.125769,1.314504,-0.577105,0.490121,-0.513071,1.057313,-0.100488,0.937130,1.036782
1,-0.320863,-1.221515,-0.022905,-0.018727,-1.330582,-0.525997,-0.482261,-0.346628,0.308145,-0.872584,-0.513071,-0.539627,-0.181448,-0.359188,-0.257860
2,1.314402,-0.333220,1.504935,1.470414,-0.242004,0.859757,1.082557,1.242281,1.193395,-0.872584,0.414735,1.004676,0.163919,1.217729,1.331764
3,1.496098,-0.333220,1.504935,1.470414,-0.242004,0.859757,1.604163,1.458950,1.193395,-0.872584,0.414735,1.185025,0.423600,1.439661,1.331764
4,-0.684255,-0.333220,-0.022905,0.477653,-0.242004,-0.064079,-0.482261,-0.779966,0.308145,-0.872584,-0.513071,-0.643077,-0.570950,-0.772831,0.432429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39462,-1.047647,-1.221515,-1.041464,-0.018727,-1.330582,-0.525997,-1.003867,-0.924413,-1.462354,1.852825,-0.513071,-0.718858,-0.339028,-0.872078,-1.126417
39463,-0.502559,0.555074,-0.532184,-0.515108,0.846574,-0.525997,1.082557,-0.491074,-0.577105,-0.872584,-1.440877,-0.474814,0.244440,-0.210518,-0.294233
39464,-1.411039,-0.333220,-1.550744,-1.011488,-0.242004,-1.449832,-1.003867,-1.502197,-0.577105,-0.872584,-1.440877,-1.401349,0.033912,-1.213474,-1.311081
39465,-0.320863,-0.333220,-0.022905,-0.515108,-0.242004,-0.525997,-1.003867,-0.129958,-0.577105,0.490121,-0.513071,-0.474210,-1.130640,-0.187839,-0.772679


In [10]:
normal_descriptors_transformer = Pipeline([('gen', descriptors_transformer), ('norm', scaler)])
X_norm_SS = normal_descriptors_transformer.fit_transform(molecules)
X_norm_SS

array([[ 0.9510099 ,  0.55507431,  0.99565539, ..., -0.10048849,
         0.93712972,  1.03678238],
       [-0.32086277, -1.2215146 , -0.0229045 , ..., -0.18144835,
        -0.35918833, -0.2578599 ],
       [ 1.31440209, -0.33322014,  1.50493534, ...,  0.16391913,
         1.21772897,  1.33176416],
       ...,
       [-1.41103935, -0.33322014, -1.55074434, ...,  0.03391206,
        -1.21347425, -1.31108077],
       [-0.32086277, -0.33322014, -0.0229045 , ..., -1.13063976,
        -0.18783893, -0.77267906],
       [-0.86595106,  1.44336877, -0.53218445, ..., -0.43051873,
        -0.79262585,  0.57832153]])

In [11]:
kf = KFold(n_splits=5, random_state=1, shuffle=True)

In [12]:
X_norm_SS = pd.DataFrame(X_norm_SS)

In [13]:
X_norm_SS.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.951010,0.555074,0.995655,0.477653,0.846574,0.859757,2.125769,1.314504,-0.577105,0.490121,-0.513071,1.057313,-0.100488,0.937130,1.036782
1,-0.320863,-1.221515,-0.022905,-0.018727,-1.330582,-0.525997,-0.482261,-0.346628,0.308145,-0.872584,-0.513071,-0.539627,-0.181448,-0.359188,-0.257860
2,1.314402,-0.333220,1.504935,1.470414,-0.242004,0.859757,1.082557,1.242281,1.193395,-0.872584,0.414735,1.004676,0.163919,1.217729,1.331764
3,1.496098,-0.333220,1.504935,1.470414,-0.242004,0.859757,1.604163,1.458950,1.193395,-0.872584,0.414735,1.185025,0.423600,1.439661,1.331764
4,-0.684255,-0.333220,-0.022905,0.477653,-0.242004,-0.064079,-0.482261,-0.779966,0.308145,-0.872584,-0.513071,-0.643077,-0.570950,-0.772831,0.432429
